# Importing Library

In [200]:
import warnings
import datetime
import numpy as np
import pandas as pd
import ipywidgets as widgets


warnings.filterwarnings("ignore")
pd.set_option('display.width', 1000)
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)

ingiltere = pd.read_csv("E0.csv")

df = ingiltere.iloc[:,1:24]
#Drop unnecessary columns
drop_elements = ['Referee']
df = df.drop(drop_elements, axis=1)

In [201]:
df.head()

,Date,Time,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HTHG,HTAG,HTR,HS,AS,HST,AST,HF,AF,HC,AC,HY,AY,HR,AR
0,12/09/2020,12:30,Fulham,Arsenal,0,3,A,0,1,A,5,13,2,6,12,12,2,3,2,2,0,0
1,12/09/2020,15:00,Crystal Palace,Southampton,1,0,H,1,0,H,5,9,3,5,14,11,7,3,2,1,0,0
2,12/09/2020,17:30,Liverpool,Leeds,4,3,H,3,2,H,22,6,6,3,9,6,9,0,1,0,0,0
3,12/09/2020,20:00,West Ham,Newcastle,0,2,A,0,0,D,15,15,3,2,13,7,8,7,2,2,0,0
4,13/09/2020,14:00,West Brom,Leicester,0,3,A,0,0,D,7,13,1,7,12,9,2,5,1,1,0,0


In [202]:
df.shape

(58, 22)

In [203]:
columns = ['Amount', 'brandid_MASTER', 'brandid_TROY', 'brandid_VISA', 'MerchantId',
         'hour', 'week_day', 'weekend', 'daynight']

# Data Dictionary

In [204]:
#Div = League Division
#Date = Match Date (dd/mm/yy)
#Time = Time of match kick off
#HomeTeam = Home Team
#AwayTeam = Away Team
#FTHG = Full Time Home Team Goals
#FTAG = Full Time Away Team Goals
#FTR = Full Time Result (H=Home Win, D=Draw, A=Away Win)
#HTHG = Half Time Home Team Goals
#HTAG = Half Time Away Team Goals
#HTR = Half Time Result (H=Home Win, D=Draw, A=Away Win)
#HS = Home Team Shots
#AS = Away Team Shots
#HST = Home Team Shots on Target
#AST = Away Team Shots on Target
#HF = Home Team Fouls Committed
#AF = Away Team Fouls Committed
#HC = Home Team Corners
#AC = Away Team Corners
#HY = Home Team Yellow Cards
#AY = Away Team Yellow Cards
#HR = Home Team Red Cards
#AR = Away Team Red Cards

In [205]:
#colunm name check

# Data Preprocessing

In [206]:
def feature_engineering(df):
    df['half_time_result_sum'] = df['HTHG'] + df['HTAG']
    df['final_result_sum'] = df['FTHG'] + df['FTAG']
    df['home_team_proportion'] = (df['HST'] / df['HS'])*100
    df['away_team_proportion'] = (df['AST'] / df['AS'])*100  
    return df
#1
df = feature_engineering(df)


def convert_str_to_date(row):
    """
    This convert date colum to datetime format
    :param row: string data in order to cnvert datetime format
    :return: year-month-day for mat of date
    """
    return datetime.datetime.strptime(row.replace('/', '-'), '%d-%m-%Y')

#df['Date'] = df['Date'].apply(lambda x: convert_str_to_date(x))
#df['month'] = df['Date'].apply(lambda x: x.month) 
#df['year'] = df['Date'].apply(lambda x: x.year)

def zero_5_Under_Half(row):
    if row['HTHG'] == 0 and row['HTAG'] == 0:
        return 1 
    else:
        return 0 

def one_5_Under_Half(row):
    if row['HTHG'] < 2 and row['HTAG'] < 2:
        return 1 
    else:
        return 0 
    
def two_5_Under_Half(row):
    if row['HTHG'] < 3 and row['HTAG'] < 3:
        return 1 
    else:
        return 0 
#2
df['0.5_under_half'] = df.apply(lambda row : zero_5_Under_Half(row),axis=1)
df['1.5_under_half'] = df.apply(lambda row : one_5_Under_Half(row),axis=1)
df['2.5_under_half'] = df.apply(lambda row : two_5_Under_Half(row),axis=1)

def zero_5_Under_Final(score):
    if score > 0:
        return 0 #bitmemıs
    else:
        return 1 #bitmis

def one_5_Under_Final(score):
    if score > 1:
        return 0 #bitmemıs
    else:
        return 1 #bitmis
    
def two_5_Under_Final(score):
    if score > 2:
        return 0 #bitmemıs
    else:
        return 1 #bitmis

def three_5_Under_Final(score):
    if score > 3:
        return 0 #bitmemıs
    else:
        return 1 #bitmis

def four_5_Under_Final(score):
    if score > 4:
        return 0 #bitmemıs
    else:
        return 1 #bitmis
    
df['0.5_under_final'] = df['final_result_sum'].apply(zero_5_Under_Final)
df['1.5_under_final'] = df['final_result_sum'].apply(one_5_Under_Final)
df['2.5_under_final'] = df['final_result_sum'].apply(two_5_Under_Final)
df['3.5_under_final'] = df['final_result_sum'].apply(three_5_Under_Final)
df['4.5_under_final'] = df['final_result_sum'].apply(four_5_Under_Final)

def zero_5_Above_Half(row):
    if row['HTHG'] > 0 or row['HTAG'] > 0:
        return 1 #
    else:
        return 0 #

def one_5_Above_Half(row):
    if row['HTHG'] > 1 or row['HTAG'] > 1:
        return 1 #
    else:
        return 0 #
    
def two_5_Above_Half(row):
    if row['HTHG'] > 2 or row['HTAG'] > 2:
        return 1 #
    else:
        return 0 #

    
df['0.5_above_half'] = df.apply(lambda row : zero_5_Above_Half(row),axis=1)
df['1.5_above_half'] = df.apply(lambda row : one_5_Above_Half(row),axis=1)
df['2.5_above_half'] = df.apply(lambda row : two_5_Above_Half(row),axis=1)

def zero_5_Above_Final(score):
    if score > 0:
        return 1 #
    else:
        return 0 #

def one_5_Above_Final(score):
    if score > 1:
        return 1 #
    else:
        return 0 #
    
def two_5_Above_Final(score):
    if score > 2:
        return 1 #
    else:
        return 0 #

def three_5_Above_Final(score):
    if score > 3:
        return 1 #
    else:
        return 0 #

def four_5_Above_Final(score):
    if score > 4:
        return 1 #
    else:
        return 0 #
    
df['0.5_above_final'] = df['final_result_sum'].apply(zero_5_Above_Final)
df['1.5_above_final'] = df['final_result_sum'].apply(one_5_Above_Final)
df['2.5_above_final'] = df['final_result_sum'].apply(two_5_Above_Final)
df['3.5_above_final'] = df['final_result_sum'].apply(three_5_Above_Final)
df['4.5_above_final'] = df['final_result_sum'].apply(four_5_Above_Final)


s1 = pd.to_numeric(df['FTHG'], errors='coerce')
s2 = pd.to_numeric(df['FTAG'], errors='coerce')

m1 = (s1 >0) & (s2 >0)
m2 = (s1 <0) & (s2 <0)

masks=[m1,m2]
vals = [1,0]

df['mutual_goal'] = np.select(masks,vals)

# Change to categoric column to numeric
df.loc[df['FTR']=='H','FTR']=1
df.loc[df['FTR']=='D','FTR']=0
df.loc[df['FTR']=='A','FTR']=2

# Change to categoric column to numeric
df.loc[df['HTR']=='H','HTR']=1
df.loc[df['HTR']=='D','HTR']=0
df.loc[df['HTR']=='A','HTR']=2

#Maç Sonucu ve Alt / Üst (1,5)

def MS_1_under_1_5(row):
    if row['FTR'] == 1 and row['final_result_sum'] < 2 :
        return 1
    else:
        return 0
    
def MS_2_under_1_5(row):
    if row['FTR'] == 2 and row['final_result_sum'] < 2 :
        return 1
    else:
        return 0
    
def MS_0_under_1_5(row):
    if row['FTR'] == 0 and row['final_result_sum'] < 2 :
        return 1
    else:
        return 0
    
def MS_1_above_1_5(row):
    if row['FTR'] == 1 and row['final_result_sum'] > 2 :
        return 1
    else:
        return 0
    
def MS_2_above_1_5(row):
    if row['FTR'] == 2 and row['final_result_sum'] > 2 :
        return 1
    else:
        return 0
    
def MS_0_above_1_5(row):
    if row['FTR'] == 0 and row['final_result_sum'] > 2 :
        return 1
    else:
        return 0
    
df['MS_1_under_1_5'] = df.apply(lambda row : MS_1_under_1_5(row), axis=1)
df['MS_2_under_1_5'] = df.apply(lambda row : MS_2_under_1_5(row), axis=1)
df['MS_0_under_1_5'] = df.apply(lambda row : MS_0_under_1_5(row), axis=1)

df['MS_1_above_1_5'] = df.apply(lambda row : MS_1_above_1_5(row), axis=1)
df['MS_2_above_1_5'] = df.apply(lambda row : MS_2_above_1_5(row), axis=1)
df['MS_0_above_1_5'] = df.apply(lambda row : MS_0_above_1_5(row), axis=1)

#Maç Sonucu ve Alt / Üst (2,5)

def MS_1_under_2_5(row):
    if row['FTR'] == 1 and row['final_result_sum'] < 3 :
        return 1
    else:
        return 0
    
def MS_2_under_2_5(row):
    if row['FTR'] == 2 and row['final_result_sum'] < 3 :
        return 1
    else:
        return 0
    
def MS_0_under_2_5(row):
    if row['FTR'] == 0 and row['final_result_sum'] < 3 :
        return 1
    else:
        return 0
    
def MS_1_above_2_5(row):
    if row['FTR'] == 1 and row['final_result_sum'] > 3 :
        return 1
    else:
        return 0
    
def MS_2_above_2_5(row):
    if row['FTR'] == 2 and row['final_result_sum'] > 3 :
        return 1
    else:
        return 0
    
def MS_0_above_2_5(row):
    if row['FTR'] == 0 and row['final_result_sum'] > 3 :
        return 1
    else:
        return 0
    
df['MS_1_under_2_5'] = df.apply(lambda row : MS_1_under_2_5(row), axis=1)
df['MS_2_under_2_5'] = df.apply(lambda row : MS_2_under_2_5(row), axis=1)
df['MS_0_under_2_5'] = df.apply(lambda row : MS_0_under_2_5(row), axis=1)

df['MS_1_above_2_5'] = df.apply(lambda row : MS_1_above_2_5(row), axis=1)
df['MS_2_above_2_5'] = df.apply(lambda row : MS_2_above_2_5(row), axis=1)
df['MS_0_above_2_5'] = df.apply(lambda row : MS_0_above_2_5(row), axis=1)

#Maç Sonucu ve Alt / Üst (3,5)

def MS_1_under_3_5(row):
    if row['FTR'] == 1 and row['final_result_sum'] < 4 :
        return 1
    else:
        return 0
    
def MS_2_under_3_5(row):
    if row['FTR'] == 2 and row['final_result_sum'] < 4 :
        return 1
    else:
        return 0
    
def MS_0_under_3_5(row):
    if row['FTR'] == 0 and row['final_result_sum'] < 4 :
        return 1
    else:
        return 0
    
def MS_1_above_3_5(row):
    if row['FTR'] == 1 and row['final_result_sum'] > 4 :
        return 1
    else:
        return 0
    
def MS_2_above_3_5(row):
    if row['FTR'] == 2 and row['final_result_sum'] > 4 :
        return 1
    else:
        return 0
    
def MS_0_above_3_5(row):
    if row['FTR'] == 0 and row['final_result_sum'] > 4 :
        return 1
    else:
        return 0
    
df['MS_1_under_3_5'] = df.apply(lambda row : MS_1_under_3_5(row), axis=1)
df['MS_2_under_3_5'] = df.apply(lambda row : MS_2_under_3_5(row), axis=1)
df['MS_0_under_3_5'] = df.apply(lambda row : MS_0_under_3_5(row), axis=1)

df['MS_1_above_3_5'] = df.apply(lambda row : MS_1_above_3_5(row), axis=1)
df['MS_2_above_3_5'] = df.apply(lambda row : MS_2_above_3_5(row), axis=1)
df['MS_0_above_3_5'] = df.apply(lambda row : MS_0_above_3_5(row), axis=1)

#Maç Sonucu ve Alt / Üst (4,5)
def MS_1_under_4_5(row):
    if row['FTR'] == 1 and row['final_result_sum'] < 5 :
        return 1
    else:
        return 0
    
def MS_2_under_4_5(row):
    if row['FTR'] == 2 and row['final_result_sum'] < 5 :
        return 1
    else:
        return 0
    
def MS_0_under_4_5(row):
    if row['FTR'] == 0 and row['final_result_sum'] < 5 :
        return 1
    else:
        return 0
    
def MS_1_above_4_5(row):
    if row['FTR'] == 1 and row['final_result_sum'] > 5 :
        return 1
    else:
        return 0
    
def MS_2_above_4_5(row):
    if row['FTR'] == 2 and row['final_result_sum'] > 5 :
        return 1
    else:
        return 0
    
def MS_0_above_4_5(row):
    if row['FTR'] == 0 and row['final_result_sum'] > 5 :
        return 1
    else:
        return 0
    
df['MS_1_under_4_5'] = df.apply(lambda row : MS_1_under_4_5(row), axis=1)
df['MS_2_under_4_5'] = df.apply(lambda row : MS_2_under_4_5(row), axis=1)
df['MS_0_under_4_5'] = df.apply(lambda row : MS_0_under_4_5(row), axis=1)

df['MS_1_above_4_5'] = df.apply(lambda row : MS_1_above_4_5(row), axis=1)
df['MS_2_above_4_5'] = df.apply(lambda row : MS_2_above_4_5(row), axis=1)
df['MS_0_above_4_5'] = df.apply(lambda row : MS_0_above_4_5(row), axis=1)

def home_goal(row,team):
    if row['HomeTeam'] == team and row['FTHG'] > 0 :
        print("home_goal: ",row['FTHG'])
        return row['FTHG']
    else:
        return 0
    
def away_goal(row,team):
    if row['AwayTeam'] == team and row['FTAG'] > 0 :
        return row['FTAG']
    else:
        return 0
############################################################
def home_goal_concede(row,team):
    if row['HomeTeam'] == team and row['FTAG'] > 0 :
        print("home_goal_concede: ",row['FTAG'])
        return row['FTAG']
    else:
        return 0

def away_goal_concede(row,team):
    if row['AwayTeam'] == team and row['FTAG'] > 0 :
        return row['FTHG']
    else:
        return 0
############################################################
def home_win_final(row,team):
    if row['FTR'] == 1 and row['HomeTeam'] == team:
        return 1
    else:
        return 0
    
def home_lose_final(row,team):
    if row['FTR'] == 2 and row['HomeTeam'] == team:
        return 1
    else:
        return 0
    
def home_draw_final(row,team):
    if row['FTR'] == 0 and row['HomeTeam'] == team:
        return 1
    else:
        return 0   

def away_draw_final(row,team):
    if row['FTR'] == 0 and row['AwayTeam'] == team:
        return 1
    else:
        return 0   

def away_win_final(row,team):
    if row['FTR'] == 2 and row['AwayTeam'] == team:
        return 1
    else:
        return 0
    
def away_lose_final(row,team):
    if row['FTR'] == 1 and row['AwayTeam'] == team:
        return 1
    else:
        return 0

def home_win_half(row,team):
    if row['HTR'] == 1 and row['HomeTeam'] == team:
        return 1
    else:
        return 0
    
def home_lose_half(row,team):
    if row['HTR'] == 2 and row['HomeTeam'] == team:
        return 1
    else:
        return 0
    
def home_draw_half(row,team):
    if row['HTR'] == 0 and row['HomeTeam'] == team:
        return 1
    else:
        return 0   

def away_draw_half(row,team):
    if row['HTR'] == 0 and row['AwayTeam'] == team:
        return 1
    else:
        return 0   

def away_win_half(row,team):
    if row['HTR'] == 2 and row['AwayTeam'] == team:
        return 1
    else:
        return 0
    
def away_lose_half(row,team):
    if row['HTR'] == 1 and row['AwayTeam'] == team:
        return 1
    else:
        return 0

def probability(df,team):
    
    df = df[(df['HomeTeam'] == team) | (df['AwayTeam'] == team)]
    oynadigi_mac = df.shape[0]
    #
    df['home_goal'] = df.apply(lambda row : home_goal(row,team), axis=1)
    df['away_goal'] = df.apply(lambda row : away_goal(row,team), axis=1)
    df['sum_goal'] = df['home_goal'] + df['away_goal'] 
    
    ################################################################################
    
    df['home_goal_concede'] = df.apply(lambda row : home_goal_concede(row,team), axis=1)
    df['away_goal_concede'] = df.apply(lambda row : away_goal_concede(row,team), axis=1)
    df['sum_concede'] = df['home_goal_concede'] + df['away_goal_concede']
    ################################################################################
    
    
    display(df.head())
            
    #
    df['home_win_final'] = df.apply(lambda row : home_win_final(row,team), axis=1)
    df['home_lose_final'] = df.apply(lambda row : home_lose_final(row,team), axis=1)
    df['home_draw_final'] = df.apply(lambda row : home_draw_final(row,team), axis=1)
    df['away_draw_final'] = df.apply(lambda row : away_draw_final(row,team), axis=1)
    df['away_win_final'] = df.apply(lambda row : away_win_final(row,team), axis=1)
    df['away_lose_final'] = df.apply(lambda row : away_lose_final(row,team), axis=1)
    #
    df['home_win_half'] = df.apply(lambda row : home_win_final(row,team), axis=1)
    df['home_lose_half'] = df.apply(lambda row : home_lose_final(row,team), axis=1)
    df['home_draw_half'] = df.apply(lambda row : home_draw_final(row,team), axis=1)
    df['away_draw_half'] = df.apply(lambda row : away_draw_final(row,team), axis=1)
    df['away_win_half'] = df.apply(lambda row : away_win_final(row,team), axis=1)
    df['away_lose_half'] = df.apply(lambda row : away_lose_final(row,team), axis=1)
    #
    df = df.iloc[:,27:].reset_index(drop=True)
    df = df.sum(axis=0).to_frame().T
    df.insert(0, 'Team', team)
    df.insert(1, 'Oynadigi_Mac', oynadigi_mac)
    
    home = df['home_goal']
    df.drop(labels=['home_goal'], axis=1,inplace = True)
    df.insert(2, 'home_goal', home)
    
    away = df['away_goal']
    df.drop(labels=['away_goal'], axis=1,inplace = True)
    df.insert(3, 'away_goal', away)
    
    home_concede = df['home_goal_concede']
    df.drop(labels=['home_goal_concede'], axis=1,inplace = True)
    df.insert(4, 'home_goal_concede', home_concede)
    
    away_concede = df['away_goal_concede']
    df.drop(labels=['away_goal_concede'], axis=1,inplace = True)
    df.insert(5, 'away_goal_concede', away_concede)
    
    ############################################################################
    
    sum = df['sum_goal']
    df.drop(labels=['sum_goal'], axis=1,inplace = True)
    df.insert(6, 'attigi_goal_ortalama', sum)
    
    sum_concede = df['sum_concede']
    df.drop(labels=['sum_concede'], axis=1,inplace = True)
    df.insert(7, 'concede_goal_ortalama', sum_concede)
    
    cols_to_keep = df.columns[6:].values.tolist()
    appended_data = []
   
    for i in df.columns[6:8]:
        #print(i)
        appended_data.append((df[i].loc[0]/oynadigi_mac))
    #print("------")
    for i in df.columns[8:]:
        #print(i)
        appended_data.append((df[i].loc[0]/oynadigi_mac)*100)    
         
    df = pd.DataFrame(appended_data).T
    temp = df.shape[1]
    #print(temp)

    df.columns = cols_to_keep
    df.insert(0, 'team', team)
    df.insert(1, 'match_played', oynadigi_mac)
    df.insert(2, 'home_goal', home)
    df.insert(3, 'away_goal', away)
    df.insert(4, 'sum_goal', sum)
    df.insert(5, 'concede_goal_home', home_concede) 
    df.insert(6, 'concede_goal_away', away_concede) 
    df.insert(7, 'concede_sum', sum_concede) 

    
    df_temp = df.iloc[:,10:temp].T
    df_temp = df_temp.sort_values(by=0,ascending = False).T
    
    df_temp.insert(0, 'team', team)
    df_temp.insert(1, 'match_played', oynadigi_mac)
    df_temp.insert(2, 'home_goal', home)
    df_temp.insert(3, 'away_goal', away)
    df_temp.insert(4, 'sum_goal', sum)
    df_temp.insert(5, 'concede_goal_home', home_concede) 
    df_temp.insert(6, 'concede_goal_away', away_concede) 
    df_temp.insert(7, 'concede_sum', sum_concede) 
    
    df_temp = df_temp.T
    df_temp = df_temp.rename(columns={0: 'Information Card'})
    
    
    #win and lose percentage
    
    
    return df_temp

In [207]:
team = 'Aston Villa'
df_result = probability(df,team).T
df_result

home_goal:  1
home_goal:  7
home_goal_concede:  2
home_goal_concede:  3


,Date,Time,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HTHG,HTAG,HTR,HS,AS,HST,AST,HF,AF,HC,AC,HY,AY,HR,AR,half_time_result_sum,final_result_sum,home_team_proportion,away_team_proportion,0.5_under_half,1.5_under_half,2.5_under_half,0.5_under_final,1.5_under_final,2.5_under_final,3.5_under_final,4.5_under_final,0.5_above_half,1.5_above_half,2.5_above_half,0.5_above_final,1.5_above_final,2.5_above_final,3.5_above_final,4.5_above_final,mutual_goal,MS_1_under_1_5,MS_2_under_1_5,MS_0_under_1_5,MS_1_above_1_5,MS_2_above_1_5,MS_0_above_1_5,MS_1_under_2_5,MS_2_under_2_5,MS_0_under_2_5,MS_1_above_2_5,MS_2_above_2_5,MS_0_above_2_5,MS_1_under_3_5,MS_2_under_3_5,MS_0_under_3_5,MS_1_above_3_5,MS_2_above_3_5,MS_0_above_3_5,MS_1_under_4_5,MS_2_under_4_5,MS_0_under_4_5,MS_1_above_4_5,MS_2_above_4_5,MS_0_above_4_5,home_goal,away_goal,sum_goal,home_goal_concede,away_goal_concede,sum_concede
16,21/09/2020,18:00,Aston Villa,Sheffield United,1,0,1,0,0,0,18,4,2,1,12,13,10,4,3,1,0,1,0,1,11.111111,25.000000,1,1,1,0,1,1,1,1,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,1,0,1,0,0,0
26,28/09/2020,17:45,Fulham,Aston Villa,0,3,2,0,2,2,16,12,3,5,18,14,5,3,6,3,0,0,2,3,18.750000,41.666667,0,0,1,0,0,0,1,1,1,1,0,1,1,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,3,3,0,0,0
37,04/10/2020,19:15,Aston Villa,Liverpool,7,2,1,4,1,1,18,14,11,8,7,10,2,7,2,1,0,0,5,9,61.111111,57.142857,0,0,0,0,0,0,0,0,1,1,1,1,1,1,1,1,1,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,1,0,0,7,0,7,2,0,2
45,18/10/2020,19:15,Leicester,Aston Villa,0,1,2,0,0,0,11,10,5,4,18,18,2,7,4,3,0,0,0,1,45.454545,40.000000,1,1,1,0,1,1,1,1,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,1,1,0,0,0
48,23/10/2020,20:00,Aston Villa,Leeds,0,3,2,0,0,0,12,27,4,9,14,17,6,5,2,2,0,0,0,3,33.333333,33.333333,1,1,1,0,0,0,1,1,0,0,0,1,1,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,3,0,3


,team,match_played,home_goal,away_goal,sum_goal,concede_goal_home,concede_goal_away,concede_sum,0.5_above_final,3.5_under_final,4.5_under_final,2.5_under_half,1.5_under_half,MS_2_under_3_5,MS_2_under_4_5,2.5_above_final,1.5_above_final,home_win_final,1.5_above_half,0.5_above_half,2.5_under_final,1.5_under_final,MS_2_above_1_5,MS_1_above_4_5,MS_1_under_4_5,MS_1_above_3_5,home_lose_final,MS_1_under_3_5,MS_1_above_2_5,MS_2_under_2_5,MS_1_under_2_5,MS_1_above_1_5,2.5_above_half,MS_2_under_1_5,MS_1_under_1_5,3.5_above_final,mutual_goal,4.5_above_final,MS_0_above_1_5,MS_0_under_4_5,home_draw_final,0.5_under_final,MS_0_above_4_5,MS_2_above_4_5,MS_0_above_3_5,MS_0_under_2_5,MS_2_above_3_5,MS_0_under_3_5,MS_0_under_1_5,MS_0_above_2_5,MS_2_above_2_5,away_draw_final
Information Card,Aston Villa,5,8,4,12,5,0,5,100,80,80,80,60,60,60,60,60,40,40,40,40,40,40,20,20,20,20,20,20,20,20,20,20,20,20,20,20,20,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [208]:
ALL = 'ALL'

def unique_sorted_values_plus_ALL(array):
    unique = array.unique().tolist()
    unique.sort()
    unique.insert(0, ALL)
    return unique

dropdown_team_1 = widgets.Dropdown(options =  unique_sorted_values_plus_ALL(df.HomeTeam))
dropdown_team_2 = widgets.Dropdown(options =  unique_sorted_values_plus_ALL(df.HomeTeam))

output_team_1 = widgets.Output()
output_team_2 = widgets.Output()

#sort = False

def dropdown_team_eventhandler_1(change):
    output_team_1.clear_output()
    with output_team_1:
        if (change.new == ALL):
            display(df)
        else:
            display(probability(df,change.new))

def dropdown_team_eventhandler_2(change):
    output_team_2.clear_output()
    with output_team_2:
        if (change.new == ALL):
            display(df)
        else:
            display(probability(df,change.new))
            
            
dropdown_team_1.observe(dropdown_team_eventhandler_1, names='value')
dropdown_team_2.observe(dropdown_team_eventhandler_2, names='value')

In [209]:
display(dropdown_team_1)

Dropdown(options=('ALL', 'Arsenal', 'Aston Villa', 'Brighton', 'Burnley', 'Chelsea', 'Crystal Palace', 'Everto…

In [210]:
display(dropdown_team_2)

Dropdown(options=('ALL', 'Arsenal', 'Aston Villa', 'Brighton', 'Burnley', 'Chelsea', 'Crystal Palace', 'Everto…

In [211]:
display(output_team_1)

Output()

In [212]:
display(output_team_2)

Output()